In [ ]:
# !pip install nltk transformers datasets torch transformers accelerate huggingface datasets

# For colab environment

In [ ]:
import torch 
print(torch.cuda.is_available())
print(torch.__version__)

True
2.6.0+cu124


In [ ]:
# !git clone https://github.com/BernardMoy/NLP-PCL-Classification.git

In [ ]:
# %cd NLP-PCL-Classification/

In [ ]:
!nvidia-smi

Sat Feb 28 16:35:33 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.163.01             Driver Version: 550.163.01     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX TITAN X     Off |   00000000:02:00.0  On |                  N/A |
| 22%   44C    P8             32W /  250W |     447MiB /  12288MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Load train and validation data set

In [ ]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt
from collections import Counter

df = pd.read_csv('data/dontpatronizeme_pcl.tsv', sep='\t')

# Remove rows with NA labels 
df = df.dropna() 

# Add a bool_labels column for binary classification
df["bool_labels"] = df["label"] > 1   # is PCL if >1

# train val split 
train_labels = pd.read_csv('data/train_semeval_parids-labels.csv')["par_id"]
val_labels = pd.read_csv('data/dev_semeval_parids-labels.csv')["par_id"]
df_train = df[df["par_id"].isin(train_labels)]
df_val = df[df["par_id"].isin(val_labels)]


# Tokenization

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoConfig, Trainer, TrainingArguments

tokenizer = RobertaTokenizer.from_pretrained("roberta-base") 

# Create text with contextual information 
def tokenize(df): 
    text_with_context = df["text"] # df["keyword"] + "</s><s>" + df["country_code"] + "</s><s>" + df["text"]

    encoding = tokenizer(
        text_with_context.tolist(), 
        padding="max_length",   # Add padding to shorter sentences 
        max_length=256,
        truncation = True, 
        return_attention_mask = True 
    )

    return encoding

/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


# Convert to pyTorch dataset

In [ ]:
import torch 
from torch.utils.data import DataLoader, TensorDataset
from datasets import Dataset

def to_dataset(df): 
    # Obtain tokens (input_ids, attention_mask) from the dataset 
    encoding = tokenize(df) 

    # Return huggingface dataset 
    return Dataset.from_dict({
        "input_ids": encoding["input_ids"], 
        "attention_mask": encoding["attention_mask"], 
        "label": df["bool_labels"].values 
    })

In [ ]:
train_dataset = to_dataset(df_train)
val_dataset = to_dataset(df_val) 

# set to torch format 
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Training 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)

    # Calculate metrics 
    accuracy = accuracy_score(labels, predictions) 
    precision = precision_score(labels, predictions) 
    recall = recall_score(labels, predictions) 
    f1 = f1_score(labels, predictions) 

    return {
        "accuracy": accuracy, 
        "precision": precision, 
        "recall": recall, 
        "f1": f1 
    }


In [ ]:
# Load roberta sequence classification model 
config = AutoConfig.from_pretrained("roberta-base", num_labels=2)  # Binary classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", config = config)

BATCH_SIZE = 32

# Set up training arguments 
training_args = TrainingArguments(
    fp16=True, 
    num_train_epochs=5, 
    learning_rate=2e-5, 
    weight_decay=0.01,
    warmup_steps=500, 
    save_strategy="no",  # low disk space 
    load_best_model_at_end=True, 
    logging_steps=50,
    output_dir="./predictions", 
    evaluation_strategy="epoch", 
    per_device_eval_batch_size=BATCH_SIZE, 
    per_device_train_batch_size=BATCH_SIZE, 
)

# Set up trainer 
trainer = Trainer(
    model = model, 
    args = training_args, 
    train_dataset=train_dataset, 
    eval_dataset=val_dataset, 
    compute_metrics=compute_metrics
)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/vol/bitbucket/bm1325/dl_cw_1/dlvenv/lib/python3.12/site-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train() 

  0%|          | 0/1310 [00:00<?, ?it/s]

{'loss': 0.645, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.19}
{'loss': 0.3886, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.38}
{'loss': 0.3018, 'learning_rate': 6e-06, 'epoch': 0.57}
{'loss': 0.2403, 'learning_rate': 7.960000000000002e-06, 'epoch': 0.76}
{'loss': 0.233, 'learning_rate': 9.960000000000001e-06, 'epoch': 0.95}


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.2091587632894516, 'eval_accuracy': 0.9163879598662207, 'eval_precision': 0.6, 'eval_recall': 0.36180904522613067, 'eval_f1': 0.45141065830721006, 'eval_runtime': 29.3073, 'eval_samples_per_second': 71.416, 'eval_steps_per_second': 2.252, 'epoch': 1.0}
{'loss': 0.2011, 'learning_rate': 1.196e-05, 'epoch': 1.15}
{'loss': 0.2192, 'learning_rate': 1.396e-05, 'epoch': 1.34}
{'loss': 0.2195, 'learning_rate': 1.5960000000000003e-05, 'epoch': 1.53}
{'loss': 0.1911, 'learning_rate': 1.796e-05, 'epoch': 1.72}
{'loss': 0.1911, 'learning_rate': 1.9960000000000002e-05, 'epoch': 1.91}


  0%|          | 0/66 [00:00<?, ?it/s]

...

  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.276966392993927, 'eval_accuracy': 0.9288103201146679, 'eval_precision': 0.7155172413793104, 'eval_recall': 0.41708542713567837, 'eval_f1': 0.526984126984127, 'eval_runtime': 29.2528, 'eval_samples_per_second': 71.549, 'eval_steps_per_second': 2.256, 'epoch': 3.0}
{'loss': 0.1307, 'learning_rate': 1.2617283950617284e-05, 'epoch': 3.05}
{'loss': 0.0489, 'learning_rate': 1.1382716049382716e-05, 'epoch': 3.24}
{'loss': 0.1023, 'learning_rate': 1.017283950617284e-05, 'epoch': 3.44}
{'loss': 0.0656, 'learning_rate': 8.938271604938272e-06, 'epoch': 3.63}
{'loss': 0.1029, 'learning_rate': 7.703703703703704e-06, 'epoch': 3.82}


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.29277586936950684, 'eval_accuracy': 0.9211657907310081, 'eval_precision': 0.585, 'eval_recall': 0.5879396984924623, 'eval_f1': 0.5864661654135338, 'eval_runtime': 29.4656, 'eval_samples_per_second': 71.032, 'eval_steps_per_second': 2.24, 'epoch': 4.0}


RuntimeError: [enforce fail at inline_container.cc:626] . unexpected pos 385687296 vs 385687184

In [ ]:
trainer.evaluate()

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


  0%|          | 0/66 [00:00<?, ?it/s]

{'eval_loss': 0.29277586936950684, 'eval_accuracy': 0.9211657907310081, 'eval_precision': 0.585, 'eval_recall': 0.5879396984924623, 'eval_f1': 0.5864661654135338, 'eval_runtime': 28.5957, 'eval_samples_per_second': 73.193, 'eval_steps_per_second': 2.308, 'epoch': 4.0}


{'eval_loss': 0.29277586936950684,
 'eval_accuracy': 0.9211657907310081,
 'eval_precision': 0.585,
 'eval_recall': 0.5879396984924623,
 'eval_f1': 0.5864661654135338,
 'eval_runtime': 28.5957,
 'eval_samples_per_second': 73.193,
 'eval_steps_per_second': 2.308,
 'epoch': 4.0}